In [19]:
library(IRdisplay)

display_html(
'<script>  
code_show=true; 
function code_toggle() {
  if (code_show){
    $(\'div.input\').hide();
  } else {
    $(\'div.input\').show();
  }
  code_show = !code_show
}  
$( document ).ready(code_toggle);
</script>
  <form action="javascript:code_toggle()">
    <input type="submit" value="Click to toggle on/off the raw code.">
 </form>'
)

# US Affordable Housing Shortage

### Project Overview

### Data Description

In [1]:
alldata1yr <-read.csv("https://raw.githubusercontent.com/MasonPutt/4E-Capstone-Project/main/alldata1yr.csv")
alldata5yr <-read.csv("https://raw.githubusercontent.com/MasonPutt/4E-Capstone-Project/main/alldata5yr.csv")

### Modeling

In [23]:
library(lfe)
library(stargazer)
base <- felm(evict_rate ~ r_totalgap_rate+unemp_rate+r_totalgap_rate*unemp_rate | period, data=alldata5yr)
base_controls <- felm(evict_rate ~ r_totalgap_rate+unemp_rate+r_totalgap_rate*unemp_rate+metro+low_educ| period, data=alldata5yr)
suppressMessages(stargazer(base, base_controls, type="text"))


                                   Dependent variable:        
                           -----------------------------------
                                       evict_rate             
                                  (1)               (2)       
--------------------------------------------------------------
r_totalgap_rate                0.079***          0.027***     
                                (0.009)           (0.008)     
                                                              
unemp_rate                     0.098***          0.099***     
                                (0.022)           (0.021)     
                                                              
metro                                            0.012***     
                                                  (0.001)     
                                                              
low_educ                                         -0.004***    
                                                  (0.0

### Model Evaluation

### Data Visualization